In [1]:

import json
import datetime


In [2]:
# populate some animal movement test data
import random

start_time = datetime.datetime(2023, 3, 22, 13, 45, 0)
start_LLA = [-38.8214807580445, 143.55769541441353, 24]
num_entries = 50
num_animals = 2

animals = ['Sus Scrofa', 'Alauda Arvensis', 'Capra Hircus', 'Cervus Unicolour', 'Pachycephala Rufiventris', 'Strepera Graculina']

with open("movements.json", "w") as outfile:
    outfile.write("[")
    for i in range(num_animals):
        time = start_time
        LLA = start_LLA
        for j in range (num_entries):
            # Data to be written
            dictionary = {
                "timestamp": time,
                "species": animals[i],
                "animalId": i+1,
                "animalTrueLLA": LLA
            }
        
            # Serializing json
            json_object = json.dumps(dictionary, indent=4, default=str)
        
            # Writing to sample.json
            outfile.write(json_object)
            if j != (num_entries-1) or i != (num_animals-1):
                outfile.write(",")

            # Change time and move animal a little:
            time = time + datetime.timedelta(minutes=5)
            LLA[0] = LLA[0] + random.uniform(-0.0002, 0.0002)
            LLA[1] = LLA[1] + random.uniform(-0.0002, 0.0002)
    outfile.write("]")

In [3]:
# populate some event test data (classified already from engine to store)
import random
from operator import add
import base64

end_time = datetime.datetime(2023, 3, 23, 13, 45, 0)
num_entries = 6
microphones = [[-38.81442552389635, 143.5617723719617,23 ],[-38.82713112867141, 143.55173018179045, 24],[-38.826897098553374, 143.56447603854625, 24]]
# function to get a random time
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    return start + datetime.timedelta(seconds=random_second)


with open("eventsClassified.json", "w") as outfile:
    outfile.write("[")
    for i in range (num_entries):
        time = random_date(start_time, end_time)
        sensorId = random.randint(1,3)
        microphoneLLA = microphones[sensorId-1]
        animalEstLLA = list( map(add, microphoneLLA, [random.uniform(-0.001, 0.001), random.uniform(-0.001, 0.001), 0]))
        animalTrueLLA = list( map(add, animalEstLLA, [random.uniform(-0.001, 0.001), random.uniform(-0.001, 0.001), 0]))
        animalLLAUncertainty = random.randint(5, 50)
        file_name = 'audio/' + animals[i] + '.wav'

        # encoding a wav file to string
        with open(file_name, 'rb') as binary_file:
            binary_file_data = binary_file.read()
            base64_encoded_data = base64.b64encode(binary_file_data)
            base64_message = base64_encoded_data.decode('utf-8')
            audioClip = base64_message
        confidence = random.randint(60, 100)
        # Data to be written

        dictionary = {         
            "timestamp": time,
            "sensorId": sensorId,
            "species": animals[i],
            "microphoneLLA":microphoneLLA,
            "animalEstLLA": animalEstLLA,
            "animalTrueLLA": animalTrueLLA,
            "animalLLAUncertainty": animalLLAUncertainty,
            "audioClip": audioClip,
            "confidence": confidence
        }
    
        # Serializing json
        json_object = json.dumps(dictionary, indent=4, default=str)
    
        # Writing to sample.json
        outfile.write(json_object)
        if i != (num_entries-1):
            outfile.write(",")

        # Change time and move animal a little:
        time = time + datetime.timedelta(minutes=5)
        LLA[0] = LLA[0] + random.uniform(-0.0002, 0.0002)
        LLA[1] = LLA[1] + random.uniform(-0.0002, 0.0002)
    outfile.write("]")

In [4]:
# read from classified events and delete the classification labels (simulator to engine)
f = open('eventsClassified.json')
data = json.load(f)

for element in data:     
     if 'confidence' in element:     
         del element['confidence']     
     if 'species' in element:     
         del element['species']     
# Writing to events.json
with open('events.json', 'w') as f:
    json.dump(data, f)

In [5]:
# test if datasets load properly
f = open('movements.json')
data = json.load(f)
print(data[0])
f = open('events.json')
data = json.load(f)
print(data[0])
f = open('eventsClassified.json')
data = json.load(f)
print(data[0])

{'timestamp': '2023-03-22 13:45:00', 'species': 'Sus Scrofa', 'animalId': 1, 'animalTrueLLA': [-38.8214807580445, 143.55769541441353, 24]}
{'timestamp': '2023-03-22 15:49:24', 'sensorId': 1, 'microphoneLLA': [-38.81442552389635, 143.5617723719617, 23], 'animalEstLLA': [-38.81467038113257, 143.56149870303253, 23], 'animalTrueLLA': [-38.81536937922293, 143.56224606532265, 23], 'animalLLAUncertainty': 12, 'audioClip': '...'}
{'timestamp': '2023-03-22 15:49:24', 'sensorId': 1, 'species': 'Sus Scrofa', 'microphoneLLA': [-38.81442552389635, 143.5617723719617, 23], 'animalEstLLA': [-38.81467038113257, 143.56149870303253, 23], 'animalTrueLLA': [-38.81536937922293, 143.56224606532265, 23], 'animalLLAUncertainty': 12, 'audioClip': '...', 'confidence': 77}
